<a href="https://colab.research.google.com/github/felipeesser/mineracaoGithubMob/blob/main/proxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# +/- 257 repositórios por grupo


# init

In [1]:
!pip install simplejson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 139 kB 7.3 MB/s 


In [2]:
import hashlib
import requests
import simplejson
import os
import sys
from flask import Flask, request, Response

In [3]:
from ipywidgets import FileUpload, interact
@interact(files=FileUpload())
def setToken(files={}):
  global token
  if files:
    for key, values in files.items():
      token= values['content'].decode("utf-8").strip()
      print("Token carregado")

interactive(children=(FileUpload(value={}, description='Upload'), Output()), _dom_classes=('widget-interact',)…

In [4]:
import datetime
SITE="https://api.github.com/"
def tokenAuth(request):
  request.headers["User-Agent"]="Mineracao"
  request.headers["Authorization"]="token {}".format(token)
  return request

response= requests.get(SITE,auth=tokenAuth)
print("status code: ",response.status_code)
print("limite de requisições: ",response.headers["x-ratelimit-limit"])
print("número de requisições disponíveis: ",response.headers["x-ratelimit-remaining"])
print("número de requisições utilizadas: ",response.headers["x-ratelimit-used"])
print("horário para o reset do limite: ",datetime.datetime.fromtimestamp(int(response.headers["x-ratelimit-reset"])).strftime('%Y-%m-%d %H:%M:%S'))

status code:  200
limite de requisições:  5000
número de requisições disponíveis:  4999
número de requisições utilizadas:  1
horário para o reset do limite:  2022-12-19 14:19:45


# Armazenamento da população

In [5]:
import time
def paginacao(path,query,pagina,caminhos,tam):
  itemsPg=1
  while(itemsPg!=0):
    response=requests.get(SITE+path+'?q='+query+'&page='+str(pagina),auth=tokenAuth)
    data=response.json()
    if(pagina==1):
      if 'total_count' in data:
        tam.append(data['total_count'])
    if 'items' in data:
      itemsPg=len(data['items'])
      if(itemsPg!=0):
        for item in data['items']:
          if item['full_name'] not in caminhos:
            caminhos.append([item['full_name']])
        pagina+=1
    else:
      if '1000' in data['message']:
        print(data)
        return
      print(data)
      time.sleep(60)
      paginacao(path,query,pagina,caminhos,tam)
      return

# Armazenamento variaveis de estudo

In [ ]:
import time
def repPossuiQ(path,repo,query,resultado):
    response=requests.get(SITE+path+'?q=repo:'+repo[0]+'+'+query,auth=tokenAuth)
    data=response.json()
    if 'items' in data:
      if(int(data['total_count'])>0):
        resultado.append(repo)
    else:
      time.sleep(60)
      repPossuiQ(path,repo,query,resultado)

def reposPossuiQ(repos,path,query):
  tam=len(repos)
  resultado=[]
  for pos in range(tam):
    repPossuiQ(path,repos[pos],query,resultado)
    if pos!=0 and (pos%49==0) or (pos==(tam-1)):
      print(str(pos+1)+' repositórios filtrados')
  return resultado

In [15]:
import time
def repAddElem(path,repo,resultado,elem):
    response=requests.get(SITE+path+repo[0],auth=tokenAuth)
    data=response.json()
    temp=repo.copy()
    while elem:
      if elem[0] in data:
        temp.append(data[elem[0]])
        elem.pop(0)
      else:
        print(data)
        response=requests.get(SITE+path+repo[0],auth=tokenAuth)
        data=response.json()
    resultado.append(temp)

In [7]:
def reposAddElem(repos,elem):
  resultado=[]
  for repo in repos:
    temp=elem.copy()
    repAddElem('repos/',repo,resultado,temp)
  return resultado

In [18]:
import time
from bs4 import BeautifulSoup
def contaDepRepo(repo):
  resultado=0
  response= requests.get('https://github.com/'+repo+'/network/dependencies')
  if response.status_code==200:
    soup=BeautifulSoup(response.text,'html.parser')
    for s in soup.select('a.Link--primary.text-bold'):
      if s.has_attr('title'):
        if '.lock' not in s['title']:
          nbr=s.findNext('span').contents[0]
          if ',' in nbr:
            resultado+=int(nbr.replace(',',""))
          else:
            resultado+=int(nbr)
  else:
    print(response)
    time.sleep(10)
    resultado=contaDepRepo(repo)
  return resultado

def contaDepRepos(repos):
  resultado=[]
  pos=0
  tam=len(repos)
  for repo in repos:
    ndep=contaDepRepo(repo[0])
    if ndep!=0:
      temp=repo.copy()
      temp.append(ndep)
      resultado.append(temp)
    pos+=1
    if (pos%50==0) or (pos==tam):
      print(str(pos)+' repositórios filtrados')
  return resultado

In [19]:
import time
from bs4 import BeautifulSoup
def contaCommitRepo(repo):
  resultado=0
  response= requests.get('https://github.com/'+repo)
  if response.status_code==200:
    soup=BeautifulSoup(response.text,'html.parser')
    for tag in soup.select("span.d-none.d-sm-inline"):
      if len(tag.find_all("strong"))>0:
        for child in tag.find_all("strong"):
          nbr=child.get_text()
          if ',' in nbr:
            resultado=nbr.replace(',',"")
          else:
            resultado=nbr
  else:
    print(response)
    time.sleep(10)
    resultado=contaDepRepo(repo)
  return resultado
def contaCommitRepos(repos):
  resultado=[]
  pos=0
  tam=len(repos)
  for repo in repos:
    ncommit=contaCommitRepo(repo[0])
    if ncommit!=0:
      temp=repo.copy()
      temp.append(ncommit)
      resultado.append(temp)
    pos+=1
    if (pos%50==0) or (pos==tam):
      print(str(pos)+' repositórios filtrados')
  return resultado

# Definição grupos

In [ ]:
def proporcao(tam1,tam2,tamf):
  prop=tamf/(tam1[0]+tam2[0])
  tam1[0]=round(tam1[0]*prop)
  tam2[0]=tamf-tam1[0]
  return

Limite de 1000 repos por query

PWA

In [21]:
response=requests.get(SITE+'search/repositories'+'?q='+'''topic:pwa
 -topic:flutter -topic:nativescript -topic:cordova -topic:capacitor -topic:react-native -topic:xamarin sort:stars''',auth=tokenAuth)
data=response.json()
if 'total_count' in data:
  print(data['total_count'])

8124


In [22]:
CAMINHOSPWA=[]
paginacao('search/repositories','''topic:pwa
 -topic:flutter -topic:nativescript -topic:cordova -topic:capacitor -topic:react-native -topic:xamarin sort:stars''',1,CAMINHOSPWA,[])
CAMINHOSPWA=reposAddElem(CAMINHOSPWA,['size','stargazers_count'])
print(len(CAMINHOSPWA),CAMINHOSPWA)
CAMINHOSPWA=contaDepRepos(CAMINHOSPWA)
print(len(CAMINHOSPWA),CAMINHOSPWA)
CAMINHOSPWA=contaCommitRepos(CAMINHOSPWA)
print(len(CAMINHOSPWA),CAMINHOSPWA)

{'message': 'API rate limit exceeded for user ID 44467278.', 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}
{'message': 'Only the first 1000 search results are available', 'documentation_url': 'https://docs.github.com/v3/search/'}
1020 [['angular/angular', 427743, 85478], ['hoppscotch/hoppscotch', 62594, 48904], ['freeCodeCamp/devdocs', 30417, 30402], ['GoogleChrome/lighthouse', 188192, 25745], ['quasarframework/quasar', 256095, 22564], ['qier222/YesPlayMusic', 32683, 20283], ['RobinLinus/snapdrop', 3551, 14118], ['vuestorefront/vue-storefront', 91316, 10198], ['Lissy93/dashy', 315626, 8586], ['alibaba/rax', 20924, 7837], ['maxence-charriere/go-app', 210777, 6718], ['andreasbm/web-skills', 122455, 6290], ['TalAter/UpUp', 30382, 4845], ['angus-c/just', 23001, 4731], ['lauragift21/awesome-learning-resources', 420, 4708], ['preactjs/preact-cli', 8135, 4624], ['NekR/offline-plugin', 1181, 4494], ['assemble/assemble', 13050, 4125], ['he

Híbrido

cordova

In [ ]:
response=requests.get(SITE+'search/repositories'+'?q='+'''topic:cordova -topic:capacitor
-topic:flutter -topic:pwa -topic:xamarin -topic:react-native -topic:native-script sort:stars''',auth=tokenAuth)
data=response.json()
if 'total_count' in data:
  print(data['total_count'])

2394


capacitor

In [ ]:
response=requests.get(SITE+'search/repositories'+'?q='+'''topic:capacitor 
-topic:flutter -topic:reactjs -topic:react-native -topic:xamarin -topic:cordova -topic:pwa sort:stars''',auth=tokenAuth)
data=response.json()
if 'total_count' in data:
  print(data['total_count'])

542


In [ ]:
CAMINHOSHIBRIDO=[]
CAMINHOSCORDOVA=[]
CAMINHOSCAPACITOR=[]
tam1=[]
tam2=[]
paginacao('search/repositories','''topic:cordova -topic:capacitor
-topic:flutter -topic:pwa -topic:xamarin -topic:react-native -topic:native-script sort:stars''',1,CAMINHOSCORDOVA,tam1)
paginacao('search/repositories','''topic:capacitor -topic:flutter 
-topic:reactjs -topic:react-native -topic:xamarin -topic:cordova -topic:pwa sort:stars''',1,CAMINHOSCAPACITOR,tam2)
proporcao(tam1,tam2,278)
print(tam1[0],tam2[0])
CAMINHOSHIBRIDO=CAMINHOSCORDOVA[:tam1[0]]+CAMINHOSCAPACITOR[:tam2[0]]
CAMINHOSHIBRIDO=reposAddElem(CAMINHOSHIBRIDO,'size')
CAMINHOSHIBRIDO=reposAddElem(CAMINHOSHIBRIDO,'stargazers_count')
CAMINHOSHIBRIDO=contaDepRepos(CAMINHOSHIBRIDO)
CAMINHOSHIBRIDO=contaCommitRepos(CAMINHOSHIBRIDO)
print(len(CAMINHOSHIBRIDO),CAMINHOSHIBRIDO)

{'message': 'API rate limit exceeded for user ID 44467278.', 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}
{'message': 'Only the first 1000 search results are available', 'documentation_url': 'https://docs.github.com/v3/search/'}
227 51
50 repositórios filtrados
100 repositórios filtrados
150 repositórios filtrados
200 repositórios filtrados
250 repositórios filtrados
278 repositórios filtrados
50 repositórios filtrados
100 repositórios filtrados
150 repositórios filtrados
200 repositórios filtrados
250 repositórios filtrados
278 repositórios filtrados
277 [['framework7io/framework7', 102716, 16982, 923, '6665'], ['Jigsaw-Code/outline-client', 560467, 7289, 1772, '1006'], ['melonjs/melonJS', 95083, 4270, 873, '6279'], ['apache/cordova-android', 37740, 3416, 376, '4045'], ['danielsogl/awesome-cordova-plugins', 32402, 2256, 1121, '4409'], ['apache/cordova-ios', 32157, 2076, 420, '3917'], ['halfrost/vue-objccn', 67016, 2004, 1040, '1

Nativo

android

In [ ]:
response=requests.get(SITE+'search/repositories'+'?q='+'''topic:android -topic:capacitor -topic:cordova
-topic:flutter -topic:pwa -topic:react-native -topic:xamarin -topic:nativescript -topic:ios sort:stars''',auth=tokenAuth)
data=response.json()
if 'total_count' in data:
  print(data['total_count'])

87154


ios

In [ ]:
response=requests.get(SITE+'search/repositories'+'?q='+'''topic:ios -topic:capacitor -topic:cordova
-topic:flutter -topic:pwa -topic:react-native -topic:xamarin -topic:nativescript -topic:android sort:stars''',auth=tokenAuth)
data=response.json()
if 'total_count' in data:
  print(data['total_count'])

26808


In [ ]:
CAMINHOSNATIVO=[]
CAMINHOSANDROID=[]
CAMINHOSIOS=[]
tam1=[]
tam2=[]
paginacao('search/repositories','''topic:android -topic:capacitor -topic:cordova
-topic:flutter -topic:pwa -topic:react-native -topic:xamarin -topic:nativescript -topic:ios sort:stars''',1,CAMINHOSANDROID,tam1)
paginacao('search/repositories','''topic:ios -topic:capacitor -topic:cordova
-topic:flutter -topic:pwa -topic:react-native -topic:xamarin -topic:nativescript -topic:android sort:stars''',1,CAMINHOSIOS,tam2)
proporcao(tam1,tam2,278)
print(tam1[0],tam2[0])
CAMINHOSNATIVO=CAMINHOSANDROID[:tam1[0]]+CAMINHOSIOS[:tam2[0]]
CAMINHOSNATIVO=reposAddElem(CAMINHOSNATIVO,'size')
CAMINHOSNATIVO=reposAddElem(CAMINHOSNATIVO,'stargazers_count')
CAMINHOSNATIVO=contaDepRepos(CAMINHOSNATIVO)
CAMINHOSNATIVO=contaCommitRepos(CAMINHOSNATIVO)
print(len(CAMINHOSNATIVO),CAMINHOSNATIVO)

{'message': 'API rate limit exceeded for user ID 44467278.', 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}
{'message': 'Only the first 1000 search results are available', 'documentation_url': 'https://docs.github.com/v3/search/'}
{'message': 'API rate limit exceeded for user ID 44467278.', 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}
{'message': 'Only the first 1000 search results are available', 'documentation_url': 'https://docs.github.com/v3/search/'}
213 65
50 repositórios filtrados
100 repositórios filtrados
150 repositórios filtrados
200 repositórios filtrados
250 repositórios filtrados
278 repositórios filtrados
50 repositórios filtrados
100 repositórios filtrados
150 repositórios filtrados
200 repositórios filtrados
250 repositórios filtrados
278 repositórios filtrados
275 [['Genymobile/scrcpy', 3805, 73399, 0, '1809'], ['Hack-with-Github/Awesome-Hacking', 165, 58518

Cross-compilado

flutter

In [ ]:
response=requests.get(SITE+'search/repositories'+'?q='+'''topic:flutter -topic:capacitor -topic:cordova
-topic:xamarin -topic:nativescript -topic:react-native -topic:pwa sort:stars''',auth=tokenAuth)
data=response.json()
if 'total_count' in data:
  print(data['total_count'])

37749


xamarin

In [ ]:
response=requests.get(SITE+'search/repositories'+'?q='+'''topic:xamarin  -topic:capacitor -topic:cordova
-topic:flutter -topic:nativescript -topic:react-native -topic:pwa sort:stars''',auth=tokenAuth)
data=response.json()
if 'total_count' in data:
  print(data['total_count'])

3832


In [ ]:
CAMINHOSCROSS=[]
CAMINHOSFLUTTER=[]
CAMINHOSXAMARIN=[]
tam1=[]
tam2=[]
paginacao('search/repositories','''topic:flutter -topic:capacitor -topic:cordova
-topic:xamarin -topic:nativescript -topic:react-native -topic:pwa sort:stars''',1,CAMINHOSFLUTTER,tam1)
paginacao('search/repositories','''topic:xamarin  -topic:capacitor -topic:cordova
-topic:flutter -topic:nativescript -topic:react-native -topic:pwa sort:stars''',1,CAMINHOSXAMARIN,tam2)
proporcao(tam1,tam2,278)
print(tam1[0],tam2[0])
CAMINHOSCROSS=CAMINHOSFLUTTER[:tam1[0]]+CAMINHOSXAMARIN[:tam2[0]]
CAMINHOSCROSS=reposAddElem(CAMINHOSCROSS,'size')
CAMINHOSCROSS=reposAddElem(CAMINHOSCROSS,'stargazers_count')
CAMINHOSCROSS=contaDepRepos(CAMINHOSCROSS)
CAMINHOSCROSS=contaCommitRepos(CAMINHOSCROSS)
print(len(CAMINHOSCROSS),CAMINHOSCROSS)

{'message': 'API rate limit exceeded for user ID 44467278.', 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}
{'message': 'Only the first 1000 search results are available', 'documentation_url': 'https://docs.github.com/v3/search/'}
{'message': 'API rate limit exceeded for user ID 44467278.', 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}
{'message': 'Only the first 1000 search results are available', 'documentation_url': 'https://docs.github.com/v3/search/'}
252 26
50 repositórios filtrados
100 repositórios filtrados
150 repositórios filtrados
200 repositórios filtrados
250 repositórios filtrados
278 repositórios filtrados
50 repositórios filtrados
100 repositórios filtrados
150 repositórios filtrados
200 repositórios filtrados
250 repositórios filtrados
278 repositórios filtrados
278 [['flutter/flutter', 227253, 146693, 2490, 2490], ['Solido/awesome-flutter', 3414, 44125, 0, '1

Interpretado

react-native

In [ ]:
response=requests.get(SITE+'search/repositories'+'?q='+'''topic:react-native -topic:capacitor -topic:cordova
-topic:flutter -topic:pwa -topic:xamarin -topic:nativescript sort:stars''',auth=tokenAuth)
data=response.json()
if 'total_count' in data:
  print(data['total_count'])

35372


native-script

In [ ]:
response=requests.get(SITE+'search/repositories'+'?q='+'''topic:nativescript -topic:capacitor -topic:cordova
-topic:flutter -topic:pwa -topic:react-native -topic:xamarin sort:stars''',auth=tokenAuth)
data=response.json()
if 'total_count' in data:
  print(data['total_count'])

726


In [ ]:
CAMINHOSINTER=[]
CAMINHOSREACT=[]
CAMINHOSNSCRIPT=[]
tam1=[]
tam2=[]
paginacao('search/repositories','''topic:react-native -topic:capacitor -topic:cordova
-topic:flutter -topic:pwa -topic:xamarin -topic:nativescript sort:stars''',1,CAMINHOSREACT,tam1)
paginacao('search/repositories','''topic:nativescript -topic:capacitor -topic:cordova
-topic:flutter -topic:pwa -topic:react-native -topic:xamarin sort:stars''',1,CAMINHOSNSCRIPT,tam2)
proporcao(tam1,tam2,278)
CAMINHOSINTER=CAMINHOSREACT[:tam1[0]]+CAMINHOSNSCRIPT[:tam2[0]]
CAMINHOSINTER=reposAddElem(CAMINHOSINTER,'size')
CAMINHOSINTER=reposAddElem(CAMINHOSINTER,'stargazers_count')
CAMINHOSINTER=contaDepRepos(CAMINHOSINTER)
CAMINHOSINTER=contaCommitRepos(CAMINHOSINTER)
print(len(CAMINHOSINTER),CAMINHOSINTER)

{'message': 'API rate limit exceeded for user ID 44467278.', 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}
{'message': 'Only the first 1000 search results are available', 'documentation_url': 'https://docs.github.com/v3/search/'}
{'message': 'API rate limit exceeded for user ID 44467278.', 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}
50 repositórios filtrados
100 repositórios filtrados
150 repositórios filtrados
200 repositórios filtrados
250 repositórios filtrados
278 repositórios filtrados
50 repositórios filtrados
100 repositórios filtrados
150 repositórios filtrados
200 repositórios filtrados
250 repositórios filtrados
278 repositórios filtrados
278 [['justjavac/free-programming-books-zh_CN', 1236, 97281, 0, '922'], ['storybookjs/storybook', 594739, 75231, 1401, '45316'], ['enaqx/awesome-react', 2429, 52417, 0, '1932'], ['laurent22/joplin', 294743, 33147, 11263, '8367'],

# Saida


In [23]:
def escreveSaida(arquivo,vetor):
  f=open(arquivo,"w")
  f.write("nome tamanho estrelas dependencias commits")
  f.write('\n')
  for lin in vetor:
    for col in lin[0:]:
      if lin.index(col)!=(len(lin)-1):
        f.write(str(col)+" ")
      else:
        f.write(str(col))
    f.write('\n')
  f.close()

In [25]:
escreveSaida("outputpwa.txt",CAMINHOSPWA)
#escreveSaida("outputcross.txt",CAMINHOSCROSS)
#escreveSaida("outputinter.txt",CAMINHOSINTER)
#escreveSaida("outputnativo.txt",CAMINHOSNATIVO)
#escreveSaida("outputhibrido.txt",CAMINHOSHIBRIDO)